In [2]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys
path="data/"
inputname="human.fasta"
outputname="human_KSNPF"

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(m6a_benchmark_dataset)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [16]:
def KSNPF_extract_one_line(data_line):
    finally_result=[]
    alphabet='AUCG'
    Compostion_key=list(alphabet)
    one_line_feature=[]
    two_encoding=[''.join(e) for e in itertools.product(alphabet,repeat=2)]
    two=np.zeros(16)
    two_part1=np.zeros(16)
    two_part2=np.zeros(16)
    two_part3=np.zeros(16)
    two_part4=np.zeros(16)

    for index,alpha in enumerate(data_line):
        if index<len(data_line)-1:
            two[two_encoding.index("".join([alpha,data_line[index+1]]))]+=1
        if index<len(data_line)-2:
            two_part1[two_encoding.index("".join([alpha,data_line[index+2]]))]+=1
        if index<len(data_line)-3:
            two_part2[two_encoding.index("".join([alpha,data_line[index+3]]))]+=1
        if index<len(data_line)-4:
            two_part3[two_encoding.index("".join([alpha,data_line[index+4]]))]+=1
        if index<len(data_line)-5:
            two_part4[two_encoding.index("".join([alpha,data_line[index+5]]))]+=1
    two=two/(len(data_line)-1)
    two_part1=two_part1/(len(data_line)-2)
    two_part2=two_part2/(len(data_line)-3)
    two_part3=two_part3/(len(data_line)-2)
    two_part4=two_part4/(len(data_line)-3)

    finally_result.extend(two)
    finally_result.extend(two_part1)
    finally_result.extend(two_part2)
    finally_result.extend(two_part3)
    finally_result.extend(two_part4)
    return finally_result



In [17]:
def KSNPF_feature_extraction(matrix_data):
    final_feature_matrix=[KSNPF_extract_one_line(e) for e in matrix_data]
    return final_feature_matrix

In [18]:
m6a_benchmark_dataset=path+inputname
matrix_data=read_fasta_file(m6a_benchmark_dataset)
final_feature_matrix=KSNPF_feature_extraction(matrix_data)
print "feature shape:",np.array(final_feature_matrix).shape
pd.DataFrame(final_feature_matrix).to_csv(path+outputname+'.csv',header=None,index=False)

feature shape: (2260, 80)


In [19]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [21]:
"""
    cross validation
"""
classifier="SVM"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['SVMC:10.8863200014gamma:0.270014934723', 0.67831858407079648, 0.7109947643979058, 0.6008849557522123, 0.6008849557522123, 0.7557522123893805, 0.6738843630039426, 0.65131894484412478, 0.65131894484412478, 0.36099243802346354, 0.67831858407079637, 679.0, 451.0, 276.0, 854.0, 1130.0, 1130.0]]]
80


True

In [22]:
"""
    cross validation GBDT
"""
classifier="GBDT"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
GBDT = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(GBDT, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=0.1,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=0.1,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['GBDTn_estimators:103max_depth:7min_samples_split:500min_samples_leaf:80', 0.68407079646017699, 0.7092555331991952, 0.6238938053097345, 0.6238938053097345, 0.7442477876106195, 0.681418802430444, 0.66384180790960456, 0.66384180790960456, 0.37083719342833299, 0.68407079646017699, 705.0, 425.0, 289.0, 841.0, 1130.0, 1130.0]]]
80


True

In [23]:
"""
    cross validation RF
"""
classifier="RF"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
RF = RandomForestClassifier(min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(RF, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(RandomForestClassifier(n_estimators=n_estimators,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(RandomForestClassifier(n_estimators=n_estimators,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['RFn_estimators:21max_depth:7min_samples_split:100min_samples_leaf:70', 0.65044247787610621, 0.6628352490421456, 0.6123893805309735, 0.6123893805309735, 0.6884955752212389, 0.6493284059765526, 0.63661453541858337, 0.63661453541858337, 0.30176014534663276, 0.65044247787610621, 692.0, 438.0, 352.0, 778.0, 1130.0, 1130.0]]]
80


True

In [24]:
"""
    cross validation NB
"""
classifier="NB"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(GaussianNB(),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(GaussianNB(),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier,ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['NB', 0.59469026548672566, 0.6126315789473684, 0.5150442477876106, 0.5150442477876106, 0.6743362831858407, 0.5893326935860121, 0.55961538461538451, 0.55961538461538451, 0.19182990992832605, 0.59469026548672554, 582.0, 548.0, 368.0, 762.0, 1130.0, 1130.0]]]
80


True

In [25]:
"""
    cross validation
"""
classifier="LR"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
logisReg = LogisticRegression()
C_range = 10.0 ** np.arange(-4,3,1)
grid_parame = dict(C=C_range)
clf = GridSearchCV(logisReg, grid_parame, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_

y_predict=cross_val_predict(LogisticRegression(C=C),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(LogisticRegression(C=C),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['LRC:100.0', 0.68097345132743359, 0.6942070275403609, 0.6469026548672566, 0.6469026548672566, 0.7150442477876107, 0.6801205938959398, 0.66972056802565283, 0.66972056802565283, 0.36279015017473548, 0.6809734513274337, 731.0, 399.0, 322.0, 808.0, 1130.0, 1130.0]]]
80


True

In [26]:
"""
    cross validation
"""
classifier="KNN"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
knn = KNeighborsClassifier()
#设置k的范围
k_range = list(range(1,10))
leaf_range = list(range(1,2))
weight_options = ['uniform','distance']
algorithm_options = ['auto','ball_tree','kd_tree','brute']
param_gridknn = dict(n_neighbors = k_range,weights = weight_options,algorithm=algorithm_options,leaf_size=leaf_range)
clf = GridSearchCV(knn, param_gridknn, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_neighbors=clf.best_params_['n_neighbors']
weights=clf.best_params_['weights']
algorithm=clf.best_params_['algorithm']
leaf_size=clf.best_params_['leaf_size']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_

y_predict=cross_val_predict(KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,algorithm=algorithm,leaf_size=leaf_size),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,algorithm=algorithm,leaf_size=leaf_size),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_neighbors:"+str(n_neighbors)+"weights:"+str(weights)+"algorithm:"+str(algorithm)+"leaf_size:"+str(leaf_size),
            ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['KNNn_neighbors:7weights:distancealgorithm:autoleaf_size:1', 0.56991150442477878, 0.5816115702479339, 0.4982300884955752, 0.4982300884955752, 0.6415929203539823, 0.5653856183934105, 0.53670162059103899, 0.53670162059103899, 0.14128242853137743, 0.56991150442477878, 563.0, 567.0, 405.0, 725.0, 1130.0, 1130.0]]]
80


True

In [27]:
"""
    cross validation RF
"""
classifier="XGBoost"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
parameters = [{'n_estimators':range(1,10,1),
                      'max_depth':range(2,10),
                      'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                      'subsample':[0.75,0.8,0.85,0.9]
                      }]
clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
learning_rate=clf.best_params_['learning_rate']
subsample=clf.best_params_['subsample']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"learning_rate:"+str(learning_rate)+"subsample:"+str(subsample),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['XGBoostn_estimators:7max_depth:2learning_rate:0.4subsample:0.8', 0.69115044247787616, 0.7460136674259681, 0.5796460176991151, 0.5796460176991151, 0.8026548672566372, 0.6820965455066618, 0.65239043824701193, 0.65239043824701193, 0.39217731364424796, 0.69115044247787605, 655.0, 475.0, 223.0, 907.0, 1130.0, 1130.0]]]
80


True